# Create medical notes from an Audio file (patient-doctor interaction)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q --upgrade bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.8 MB/s eta 0:00:00


In [3]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [4]:
# Constants

LLAMA = "meta-llama/Llama-3.1-8B-Instruct"

In [5]:
audio_filename = "/content/drive/MyDrive/llms/doc-patient-interaction.mp3"

Download any audio file of a mock consultation/conversation between a doctor and a patient from Kaggle or any source.

In [6]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

# Open the file

audio_file = open(audio_filename, "rb")

# STEP 1: Transcribe Audio

## Option 1: Use Open Source for Transcription - Hugging Face Pipelines

In [7]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium.en",
    dtype=torch.float16,
    device='cuda',
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print(transcription)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.


 I understand that you've been having some diarrhea. Can you tell me a little bit about that? Yeah, sure. I've been having diarrhea for the past three to four days, and it's been pretty bad. I couldn't go to my classes. I had to skip because it was just embarrassing. Oh, okay, I'm glad to hear that. Three to four, have you ever experienced anything like this before? Have I ever experienced something like this before? No, no. I don't think so. Okay. And over the three to four days, have your stools all been loose or have you had some solid ones in between? I think they've mostly been loose. I think they've all been loose, actually, if I think about it. Okay, and how many times a day have you been having episodes of diarrhea? Oh my god, I've lost count. I'm going every hour. Maybe... Okay. Every hour. Okay. Okay. And have you noticed any changes in the color of your stool? No. It's brown. Okay. Any blood in your stool that you've noticed? Not really. I don't think so. I don't look that m

In [8]:
open_source_transcription = transcription

## Option 2: Use OpenAI for Transcription

In [9]:
# Sign in to OpenAI using Secrets in Colab

AUDIO_MODEL = "gpt-4o-mini-transcribe"

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

I understand that you've been having some diarrhea. Can you tell me a little bit about that? Yeah, sure. I've been having diarrhea for the past three to four days and it's been pretty bad. I couldn't go to my classes. I had to skip because it was just embarrassing. Okay, I'm sorry to hear that. Have I ever experienced something like this before? I don't think so. And over the three to four days, have your stools all been loose or have you had some solid ones in between? I think they've mostly been loose. I think they've all been loose actually, if I think about it. Okay, and how many times a day have you been having episodes of diarrhea? Oh my god, I've lost count. I'm going every hour. Maybe... Okay, every hour. Okay. And have you noticed any changes in the color of your stool? No, it's brown. Okay. Any blood in your stool that you've noticed? Not really, I don't think so. I don't look that much, to be honest. Okay. Have you noticed any pain associated with it? Well, I didn't start of

In [10]:
display(Markdown(open_source_transcription))
print("\n\n")
display(Markdown(transcription))

 I understand that you've been having some diarrhea. Can you tell me a little bit about that? Yeah, sure. I've been having diarrhea for the past three to four days, and it's been pretty bad. I couldn't go to my classes. I had to skip because it was just embarrassing. Oh, okay, I'm glad to hear that. Three to four, have you ever experienced anything like this before? Have I ever experienced something like this before? No, no. I don't think so. Okay. And over the three to four days, have your stools all been loose or have you had some solid ones in between? I think they've mostly been loose. I think they've all been loose, actually, if I think about it. Okay, and how many times a day have you been having episodes of diarrhea? Oh my god, I've lost count. I'm going every hour. Maybe... Okay. Every hour. Okay. Okay. And have you noticed any changes in the color of your stool? No. It's brown. Okay. Any blood in your stool that you've noticed? Not really. I don't think so. I don't look that much, to be honest. Okay. Have you noticed any pain associated with it? Well, I didn't start off with pain, but I do have a bit of a cramp now since yesterday. It's just, it hurts. Yeah, it hurts sometimes. It comes and goes. Okay. And do you feel that you still have control over releasing it? Does it come on and you don't even notice that it has happened, or you still are able to kind of have a warning before you get to the toilet? No, I do get a warning, but it's just I do need to get to the toilet really fast. Okay. Yeah. Okay. And before this happened, have you noticed, have you, is there anything that you've done differently, that you've been eating differently or any other changes that have happened in your life recently? I, I did go to, to, um, I did go to a new restaurant five days ago. Okay. What did you have there? Um, I, I just got some, uh, rice, uh, with, um, chicken, uh, chicken and durian. It was It was a nice little Chinese restaurant and, uh, yeah. Did anyone else have the same thing that you did that you were with? Um, I, no, not really. I went with my friend and I haven't really, I haven't talked to her about this. I never really thought that these were related, that this was related to that. And it's, you know, absolutely, potentially not, like it's maybe not, it could possibly be. So not to worry you about that, we just like to get an idea of kind of the whole picture of what you're coming in with. So in terms of the last three to four days, have you experienced any nausea or vomiting? Yeah, yeah, I have actually. But it's not been as bad as the diarrhea. I didn't even, yeah, I wasn't even thinking about that. Yeah, I did vomit, I think, twice yesterday. Okay, have you noticed any blood in the vomit? No, it's just watery. Okay, and are you nauseated all the time, or is it kind of on and off? It's on and off. Okay, have you taken any over-the-counter medicine or anything to help? Not really. I was just trying to drink a lot of water because I know that I can get dehydrated, so just drinking lots of water. Yeah, that's great. Okay, well it's good to hear. Have you been eating normally or have you not been eating as much as you regularly do? I have been eating less, especially since yesterday when I had my abdominal cramps started. I just don't feel like eating. Is there anything that has been making it worse? Making what worse? Just in general, is there anything that you're finding makes your nausea or your diarrhea come on? Not really. I don't think so. Do you notice any changes to your urine or any discomfort when going to the bathroom? No. Have you traveled anywhere recently? No. Okay. And have you noticed any fever or chills in the last few days? I've just been feeling really tired and I don't know, maybe. I haven't checked my temperature. Have you noticed any rashes or changes in your skin? No. Okay. Have you noticed any major changes to your weight? I know that you might not weigh yourself every day, but anything that jumps out to you? I actually haven't weighed myself in the past five days, so I don't know. Okay. Okay. Okay. I'm just going to jump to you and your general history. Do you have any history of problems with your digestive system? No. Okay. Do you have any chronic conditions that you see a family doctor for regularly? I have asthma. Okay. Do you take any medications? Yeah, I've got some buffers that my family dog gave me. Okay. Nothing else? that's it. Do you have any allergies? No, I don't. Have you ever been hospitalized in the past? Sorry, can you say that again? Have you ever been hospitalized in the past? Um, I had my wisdom teeth removed, but no, I wasn't hospitalized, no. Okay, have you been in contact with anyone recently that's been sick? Nope, I don't think so. Okay. This is your living situation right now. You said you're a student? I am. Okay. And do you live on your own, with family, with roommates? I live with two roommates. Do you drink alcohol? Sometimes. How many drinks per week would you say? I might have a glass or two on the weekends. Do you smoke? No. Do you use any other substances? No. And what would you eat on a regular day with your typical diet? Oh gosh, well, I like to have oatmeal for breakfast. I usually make something quick for lunch, maybe like a salad, and then I'll probably just like eat out at night, maybe get like a burger or fries or something. Okay. Cool. And I just wanted to jump over to ask you about your family history. Does anyone in your family have any history of digestive problems or abdominal problems, abdominal surgeries? My brother had appendicitis when he was young. My mom has had a hysterectomy, if that counts. Okay. Yeah. Nothing else. Okay. Okay. Thank you for sharing that with me. So, I think that's everything that I wanted to ask you for today.

I understand that you've been having some diarrhea. Can you tell me a little bit about that? Yeah, sure. I've been having diarrhea for the past three to four days and it's been pretty bad. I couldn't go to my classes. I had to skip because it was just embarrassing. Okay, I'm sorry to hear that. Have I ever experienced something like this before? I don't think so. And over the three to four days, have your stools all been loose or have you had some solid ones in between? I think they've mostly been loose. I think they've all been loose actually, if I think about it. Okay, and how many times a day have you been having episodes of diarrhea? Oh my god, I've lost count. I'm going every hour. Maybe... Okay, every hour. Okay. And have you noticed any changes in the color of your stool? No, it's brown. Okay. Any blood in your stool that you've noticed? Not really, I don't think so. I don't look that much, to be honest. Okay. Have you noticed any pain associated with it? Well, I didn't start off with pain, but I do have a bit of a cramp now since yesterday. It hurts sometimes, it comes and goes. Okay. And do you feel that you still have control over releasing it? Does it come on and you don't even notice that it has happened or you still are able to kind of have a warning before you get to the toilet? No, I do get a warning, but it's just I do need to get to the toilet really fast. Okay. And before this happened, have you noticed... Is there anything that you've done differently, that you've been eating differently or any other changes that have happened in your life recently? I did go to a new restaurant five days ago. Okay. What did you have there? I just got some rice with chicken venturian. It was a nice little Chinese restaurant. And yeah. Did anyone else have the same thing that you did, that you were with? No, not really. I went with my friend. I haven't really talked to her about this. I never really thought that these were related, that this was related to that. And it's absolutely potentially not. Like it's maybe not. It could possibly be. So not to worry you about that. We just like to get an idea of kind of the whole picture of what you're coming in with. So in terms of the last three to four days, have you experienced any nausea or vomiting? Yeah, yeah, I have actually. But it's not been as bad as the diarrhea. I wasn't even thinking about that. Yeah, I did vomit, I think, twice yesterday. Okay. Have you noticed any blood in the vomit? No, it's just watery. Okay. And are you nauseated all the time or is it kind of on and off? It's on and off. Okay. Have you taken any over-the-counter medicine or anything to help? Not really. I was just trying to drink a lot of water because I know that I can get dehydrated. So just drinking lots of water. Okay, so that's good to hear. Have you been eating normally or have you not been eating as much as you regularly do? I have been eating less, especially since yesterday when I had these abdominal cramps started. I just don't feel like eating. Okay. Is there anything that has been making it worse? Making what worse? Just in general, like is there anything that you're finding makes your nausea or your diarrhea come on? Not really. I don't think so. Have you noticed any changes to your urine or any discomfort when going to the bathroom? No. Have you traveled anywhere recently? No. Okay. And have you noticed any fever or chills in the last few days? I've just been feeling really tired. And I don't know. Maybe I haven't checked my temperature. Okay. Have you noticed any rashes or changes in your skin? No. Have you noticed any major changes to your weight? I know that you might not weigh yourself every day, but anything that jumps out to you? I actually haven't weighed myself in the past five days, so I don't know. Okay. And I'm just going to jump to you and your general history. Do you have any history of problems with your digestive system? No. Okay. Do you have any chronic conditions that you see a family doctor for regularly? I have asthma. Okay. Do you take any medications? Yeah, I've got some puffers that my family doctor gave me. Okay. Nothing else? Nope, that's it. Okay. Do you have any allergies? No, I don't. Okay. Have you ever been hospitalized in the past? Sorry, can you say that again? Have you ever been hospitalized in the past? I had my wisdom teeth removed, but no, I wasn't hospitalized, no. Okay. Have you been in contact with anyone recently that's been sick? Nope, I don't think so. Okay. Since your living situation right now, you said you're a student? I am. Okay. And do you live on your own, with family, with roommates? I live with two roommates. Do you drink alcohol? Sometimes. Okay. How many drinks per week would you say? I might have like a glass or two on the weekends. Okay. Do you smoke? No. Okay. Do you use any other substances? No. Okay. And what would you eat on a regular day? Your typical diet be? Oh, gosh. Well, I like to have oatmeal for breakfast. I usually make something quick for lunch, maybe like a salad. And then I might have... I'll probably just like eat out at night, maybe get like a burger or fries or something. Okay. Cool. And I just wanted to jump over to ask you about your family history. Does anyone in your family have any history of digestive problems or abdominal problems, abdominal surgeries? My brother had appendicitis when he was young. My mom has had a hysterectomy, if that counts. Yeah. Nothing else. Okay. Okay. Thank you for sharing that with me. So I think that's everything that I wanted to ask you for today.


# STEP 2: Analyze & Report

In [11]:
system_message = """
You are an expert Medical Scribe AI, specializing in creating structured, concise clinical documentation from raw patient-physician transcripts.

**Mandatory Instructions:**
1.  **Format:** Output must be strictly in the **SOAP (Subjective, Objective, Assessment, Plan)** format using Markdown headings.
2.  **Conciseness:** Notes must be clinically concise. Use bullet points extensively. Do not use conversational language.
3.  **Accuracy:** Extract information *only* from the provided transcript. Do not infer, speculate, or add information. If a section is not discussed, state 'N/A' or 'None discussed'.
4.  **Terminology:** Use standard medical terminology where possible.
5.  **Output Structure:** The final output must be only the SOAP note, formatted exactly as shown below, without any introductory or concluding remarks.
"""

user_prompt = f"""
Below is the transcript of a patient-physician clinical encounter.

Generate a comprehensive clinical note strictly following the SOAP format as instructed. Ensure all relevant findings, diagnoses, and actions are included under the correct section.

**Transcription to Analyze:**
{transcription}

**Required Output Structure:**
# Subjective
# Objective
# Assessment
# Plan
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [12]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert Medical Scribe AI, specializing in creating structured, concise clinical documentation from raw patient-physician transcripts.

**Mandatory Instructions:**
1.  **Format:** Output must be strictly in the **SOAP (Subjective, Objective, Assessment, Plan)** format using Markdown headings.
2.  **Conciseness:** Notes must be clinically concise. Use bullet points extensively. Do not use conversational language.
3.  **Accuracy:** Extract information *only* from the provided transcript. Do not infer, speculate, or add information. If a section is not discussed, state 'N/A' or 'None discussed'.
4.  **Terminology:** Use standard medical terminology where possible.
5.  **Output Structure:** The final output must be only the SOAP note, formatted exactly as shown below, without any introductory or concluding remarks.<|eot_id|><|start_header_id|>user<|end_heade

In [14]:
response = tokenizer.decode(outputs[0])

In [15]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert Medical Scribe AI, specializing in creating structured, concise clinical documentation from raw patient-physician transcripts.

**Mandatory Instructions:**
1.  **Format:** Output must be strictly in the **SOAP (Subjective, Objective, Assessment, Plan)** format using Markdown headings.
2.  **Conciseness:** Notes must be clinically concise. Use bullet points extensively. Do not use conversational language.
3.  **Accuracy:** Extract information *only* from the provided transcript. Do not infer, speculate, or add information. If a section is not discussed, state 'N/A' or 'None discussed'.
4.  **Terminology:** Use standard medical terminology where possible.
5.  **Output Structure:** The final output must be only the SOAP note, formatted exactly as shown below, without any introductory or concluding remarks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is the transcript of a patient-physician clinical encounter.

Generate a comprehensive clinical note strictly following the SOAP format as instructed. Ensure all relevant findings, diagnoses, and actions are included under the correct section.

**Transcription to Analyze:**
I understand that you've been having some diarrhea. Can you tell me a little bit about that? Yeah, sure. I've been having diarrhea for the past three to four days and it's been pretty bad. I couldn't go to my classes. I had to skip because it was just embarrassing. Okay, I'm sorry to hear that. Have I ever experienced something like this before? I don't think so. And over the three to four days, have your stools all been loose or have you had some solid ones in between? I think they've mostly been loose. I think they've all been loose actually, if I think about it. Okay, and how many times a day have you been having episodes of diarrhea? Oh my god, I've lost count. I'm going every hour. Maybe... Okay, every hour. Okay. And have you noticed any changes in the color of your stool? No, it's brown. Okay. Any blood in your stool that you've noticed? Not really, I don't think so. I don't look that much, to be honest. Okay. Have you noticed any pain associated with it? Well, I didn't start off with pain, but I do have a bit of a cramp now since yesterday. It hurts sometimes, it comes and goes. Okay. And do you feel that you still have control over releasing it? Does it come on and you don't even notice that it has happened or you still are able to kind of have a warning before you get to the toilet? No, I do get a warning, but it's just I do need to get to the toilet really fast. Okay. And before this happened, have you noticed... Is there anything that you've done differently, that you've been eating differently or any other changes that have happened in your life recently? I did go to a new restaurant five days ago. Okay. What did you have there? I just got some rice with chicken venturian. It was a nice little Chinese restaurant. And yeah. Did anyone else have the same thing that you did, that you were with? No, not really. I went with my friend. I haven't really talked to her about this. I never really thought that these were related, that this was related to that. And it's absolutely potentially not. Like it's maybe not. It could possibly be. So not to worry you about that. We just like to get an idea of kind of the whole picture of what you're coming in with. So in terms of the last three to four days, have you experienced any nausea or vomiting? Yeah, yeah, I have actually. But it's not been as bad as the diarrhea. I wasn't even thinking about that. Yeah, I did vomit, I think, twice yesterday. Okay. Have you noticed any blood in the vomit? No, it's just watery. Okay. And are you nauseated all the time or is it kind of on and off? It's on and off. Okay. Have you taken any over-the-counter medicine or anything to help? Not really. I was just trying to drink a lot of water because I know that I can get dehydrated. So just drinking lots of water. Okay, so that's good to hear. Have you been eating normally or have you not been eating as much as you regularly do? I have been eating less, especially since yesterday when I had these abdominal cramps started. I just don't feel like eating. Okay. Is there anything that has been making it worse? Making what worse? Just in general, like is there anything that you're finding makes your nausea or your diarrhea come on? Not really. I don't think so. Have you noticed any changes to your urine or any discomfort when going to the bathroom? No. Have you traveled anywhere recently? No. Okay. And have you noticed any fever or chills in the last few days? I've just been feeling really tired. And I don't know. Maybe I haven't checked my temperature. Okay. Have you noticed any rashes or changes in your skin? No. Have you noticed any major changes to your weight? I know that you might not weigh yourself every day, but anything that jumps out to you? I actually haven't weighed myself in the past five days, so I don't know. Okay. And I'm just going to jump to you and your general history. Do you have any history of problems with your digestive system? No. Okay. Do you have any chronic conditions that you see a family doctor for regularly? I have asthma. Okay. Do you take any medications? Yeah, I've got some puffers that my family doctor gave me. Okay. Nothing else? Nope, that's it. Okay. Do you have any allergies? No, I don't. Okay. Have you ever been hospitalized in the past? Sorry, can you say that again? Have you ever been hospitalized in the past? I had my wisdom teeth removed, but no, I wasn't hospitalized, no. Okay. Have you been in contact with anyone recently that's been sick? Nope, I don't think so. Okay. Since your living situation right now, you said you're a student? I am. Okay. And do you live on your own, with family, with roommates? I live with two roommates. Do you drink alcohol? Sometimes. Okay. How many drinks per week would you say? I might have like a glass or two on the weekends. Okay. Do you smoke? No. Okay. Do you use any other substances? No. Okay. And what would you eat on a regular day? Your typical diet be? Oh, gosh. Well, I like to have oatmeal for breakfast. I usually make something quick for lunch, maybe like a salad. And then I might have... I'll probably just like eat out at night, maybe get like a burger or fries or something. Okay. Cool. And I just wanted to jump over to ask you about your family history. Does anyone in your family have any history of digestive problems or abdominal problems, abdominal surgeries? My brother had appendicitis when he was young. My mom has had a hysterectomy, if that counts. Yeah. Nothing else. Okay. Okay. Thank you for sharing that with me. So I think that's everything that I wanted to ask you for today.


**Required Output Structure:**
# Subjective
# Objective
# Assessment
# Plan<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# Subjective
* Reports experiencing diarrhea for 3-4 days, with episodes occurring every hour
* Stools have been mostly loose, with no blood or significant changes in color
* Associated with abdominal cramps, which started yesterday
* Nausea and vomiting present, with episodes of vomiting occurring twice yesterday
* No fever, chills, or significant changes in urine or skin
* Recent change in diet, eating at a new restaurant 5 days ago
* No history of digestive problems, but has asthma and a history of wisdom teeth removal

# Objective
* N/A (no objective findings or measurements reported)

# Assessment
* Possible foodborne illness, given recent change in diet
* Gastroenteritis or viral gastroenteritis as a potential diagnosis
* Abdominal cramps and diarrhea suggest a possible gastrointestinal issue
* Nausea and vomiting may indicate a more severe gastrointestinal infection
* No significant findings to suggest a more serious underlying condition

# Plan
* Order stool test to rule out bacterial or parasitic infection
* Consider ordering blood tests to check for electrolyte imbalances and possible liver function
* Prescribe anti-diarrheal medication (e.g., loperamide) to manage symptoms
* Recommend hydration and rest
* Follow-up appointment in 2-3 days to re-evaluate symptoms and assess response to treatment<|eot_id|>